# Amazon Nova Meta Prompter

Transform any prompt to align with Amazon Nova guidelines.

This tool analyzes your existing prompts and adapts them for Amazon Nova capabilities including:
- Up to 1M token context
- Structured output formatting
- Chain-of-thought reasoning
- Clear section-based organization

## New: Intent-Based Pipeline

The meta prompter now includes automatic intent classification:
1. **Intent Classification** (Nova Micro) - Detects what your prompt needs (image understanding, RAG, tool use, etc.)
2. **Targeted Guidance Loading** - Only loads relevant prompt guidance for detected intents
3. **Optimized Transformation** (Nova Premier) - Transforms your prompt with focused best practices

## Setup

Configure AWS credentials via environment variables (recommended) or AWS CLI configuration.

**Option 1: Environment Variables (Recommended)**
```bash
import os
os.environ["AWS_PROFILE"]="your-profile-name"
os.environ["AWS_REGION"]="your-aws-region"
```

**Option 2: AWS CLI Configuration**
```bash
aws configure
```

## Verify Credentials

Test your AWS credentials and Bedrock access:

In [2]:
import boto3

try:
    # This will use AWS_PROFILE and AWS_REGION from environment
    client = boto3.client('bedrock-runtime')
    print("✅ AWS credentials configured successfully")
    print(f"   Region: {client.meta.region_name}")
except Exception as e:
    print(f"❌ AWS credentials error: {e}")
    print("   Please set AWS_PROFILE and AWS_REGION environment variables")
    print("   Or configure using 'aws configure'")

✅ AWS credentials configured successfully
   Region: us-east-1


## Imports

Import the transform function and display utilities:

In [1]:
from IPython.display import display, HTML

# Import transform functions
from transform import (
    transform_prompt,
    classify_intent,
    transform_with_intent_classification,
    VALID_INTENTS,
    API_CAPABILITY_GUIDANCE,
)

# Display helper functions
def display_section_header(title):
    display(HTML(f'<h3 style="color: #2E86AB; border-bottom: 2px solid #2E86AB; padding-bottom: 10px;">{title}</h3>'))

def display_original_prompt(prompt, title="Original Prompt"):
    display_section_header(title)
    display(HTML(f'<div style="background-color: #f8f9fa; padding: 15px; border-radius: 5px; border-left: 4px solid #6c757d;"><pre style="white-space: pre-wrap; font-family: monospace;">{prompt.strip()}</pre></div>'))

def display_intent_results(result):
    """Display intent classification and guidance loading results."""
    if 'intents' in result:
        intents_html = ', '.join([f'<span style="background-color: #e3f2fd; padding: 2px 8px; border-radius: 12px; margin: 2px;">{i}</span>' for i in result['intents']]) or '<em>None detected</em>'
        display(HTML(f'<h4 style="color: #1565C0;">🎯 Detected Intents</h4>'))
        display(HTML(f'<div style="background-color: #e3f2fd; padding: 15px; border-radius: 5px; margin-bottom: 10px;">{intents_html}</div>'))
    
    if 'api_capabilities' in result:
        enabled = [k for k, v in result['api_capabilities'].items() if v]
        if enabled:
            caps_html = ', '.join([f'<span style="background-color: #e8f5e9; padding: 2px 8px; border-radius: 12px; margin: 2px;">{c}</span>' for c in enabled])
            display(HTML(f'<h4 style="color: #2E7D32;">⚙️ API Capabilities Enabled</h4>'))
            display(HTML(f'<div style="background-color: #e8f5e9; padding: 15px; border-radius: 5px; margin-bottom: 10px;">{caps_html}</div>'))
    
    if 'intent_reasoning' in result and result['intent_reasoning']:
        display(HTML(f'<div style="background-color: #f5f5f5; padding: 10px; border-radius: 5px; margin-bottom: 10px; font-size: 0.9em;"><strong>Reasoning:</strong> {result["intent_reasoning"]}</div>'))
    
    if 'guidance_files' in result:
        files_html = ', '.join([f'<code>{f}</code>' for f in result['guidance_files']])
        display(HTML(f'<div style="background-color: #fff3e0; padding: 10px; border-radius: 5px; margin-bottom: 20px;"><strong>📚 Guidance Loaded:</strong> {files_html}</div>'))

def display_nova_migration_results(result):
    if "error" in result:
        display(HTML(f'<div style="color: red; padding: 10px; border: 1px solid red; border-radius: 5px;"><strong>Error:</strong> {result["error"]}</div>'))
        return
    
    # Show intent results if present
    if 'intents' in result:
        display_intent_results(result)
    
    if 'thinking' in result:
        display(HTML('<h4 style="color: #2E86AB;">🧠 Analysis & Thinking</h4>'))
        display(HTML(f'<div style="background-color: #f8f9fa; padding: 15px; border-radius: 5px; margin-bottom: 20px;">{result["thinking"].replace(chr(10), "<br>")}</div>'))
    
    if 'nova_draft' in result:
        display(HTML('<h4 style="color: #A23B72;">📝 Nova-Aligned Draft</h4>'))
        display(HTML(f'<div style="background-color: #fff3e0; padding: 15px; border-radius: 5px; margin-bottom: 20px; border-left: 4px solid #A23B72;"><pre style="white-space: pre-wrap; font-family: monospace;">{result["nova_draft"]}</pre></div>'))
    
    if 'reflection' in result:
        display(HTML('<h4 style="color: #F18F01;">🤔 Reflection</h4>'))
        display(HTML(f'<div style="background-color: #fff8e1; padding: 15px; border-radius: 5px; margin-bottom: 20px;">{result["reflection"].replace(chr(10), "<br>")}</div>'))
    
    if 'nova_final' in result:
        display(HTML('<h4 style="color: #C73E1D;">✨ Final Nova-Aligned Prompt</h4>'))
        display(HTML(f'<div style="background-color: #e8f5e8; padding: 15px; border-radius: 5px; border-left: 4px solid #C73E1D;"><pre style="white-space: pre-wrap; font-family: monospace; font-weight: bold;">{result["nova_final"]}</pre></div>'))

def display_error(error_message, note=None):
    display(HTML(f'<div style="color: red; padding: 10px; border: 1px solid red; border-radius: 5px;"><strong>Error:</strong> {error_message}</div>'))
    if note:
        print(f"\nNote: {note}")

print("✅ Setup complete!")
print(f"📋 Available intents: {', '.join(VALID_INTENTS)}")
print(f"⚙️ API capability flags: {', '.join(API_CAPABILITY_GUIDANCE.keys())}")

✅ Setup complete!
📋 Available intents: image_understanding, video_understanding, document_understanding, tool_use, structured_output, rag, multilingual, agentic
⚙️ API capability flags: reasoning_mode, tool_use, image, video


---

# Intent-Based Pipeline (Recommended)

The full pipeline automatically:
1. Classifies your prompt's intents using Nova Micro (fast & cheap)
2. Loads only the relevant guidance files
3. Transforms with Nova Premier using targeted best practices

In [4]:
# Example: Image understanding with structured output
example_prompt = """
Analyze this image and return a JSON object with all detected objects, 
their locations, and confidence scores
"""

display_original_prompt(example_prompt, "Example: Image + Structured Output")
print("\n🔄 Running intent-based pipeline...")


🔄 Running intent-based pipeline...


In [2]:
# Transform using the full intent-based pipeline
try:
    result = transform_with_intent_classification("Hola, cuanto cuesta")
    display_nova_migration_results(result)
except Exception as e:
    display_error(str(e), "This requires AWS credentials and access to Bedrock service.")

---

# Try Your Own Prompt

Enter your prompt below and run the cell to transform it:

In [ ]:
# Your custom prompt here
your_prompt = """
Summarize the following Document: {document_text}
"""

display_original_prompt(your_prompt, "Your Prompt")
print("\n🔄 Running intent-based pipeline...")

In [ ]:
# Transform your prompt
try:
    your_result = transform_with_intent_classification(your_prompt.strip())
    display_nova_migration_results(your_result)
except Exception as e:
    display_error(str(e), "This requires AWS credentials and access to Bedrock service.")

---

# More Examples by Intent Type

Try these example prompts to see how different intents are detected:

In [3]:
# Example prompts for different intent combinations
example_prompts = {
    "RAG + Multilingual": """
Based on the following context documents, answer the user's question in Spanish.
Context: {retrieved_documents}
Question: {user_question}
""",
    "Video Understanding": """
Analyze this security footage and identify all people entering the building,
noting timestamps for each entry: {video}
""",
    "Agentic + Tool Use": """
You are a research assistant. Complete the following task step by step:
1. Search for recent papers on {topic}
2. Summarize the top 3 findings
3. Identify gaps in the research
Use the search_papers and summarize tools as needed.
""",
    "Document + Structured Output": """
Extract all invoice line items from this PDF and return them as a JSON array
with fields: item_name, quantity, unit_price, total: {invoice_pdf}
"""
}

# Display all examples
for name, prompt in example_prompts.items():
    display_original_prompt(prompt, f"Example: {name}")

In [4]:
# Quick intent classification for each example (without full transformation)
print("🎯 Intent Classification Results:\n")

for name, prompt in example_prompts.items():
    try:
        intent_result = classify_intent(prompt.strip())
        intents = intent_result.get('intents', [])
        print(f"📌 {name}:")
        print(f"   Intents: {', '.join(intents) if intents else 'None'}")
        print()
    except Exception as e:
        print(f"📌 {name}: Error - {e}\n")

🎯 Intent Classification Results:

📌 RAG + Multilingual:
   Intents: multilingual, rag

📌 Video Understanding:
   Intents: video_understanding, structured_output

📌 Agentic + Tool Use:
   Intents: tool_use, agentic

📌 Document + Structured Output:
   Intents: document_understanding, structured_output



---

# Summary

## What the Meta Prompter Does

1. **Intent Classification** - Automatically detects prompt capabilities needed:
   - `image_understanding`, `video_understanding`, `document_understanding`
   - `tool_use`, `structured_output`, `rag`
   - `multilingual`, `agentic`

2. **API Capability Flags** - Explicitly enable API-level features:
   - `reasoning_mode=True` - Enable reasoning mode guidance
   - `tool_use=True` - Enable tool use guidance
   - `image=True` - Enable image understanding guidance
   - `video=True` - Enable video understanding guidance

3. **Targeted Guidance Loading** - Only loads relevant Nova best practices

4. **Prompt Transformation** - Applies Nova guidelines:
   - Clear section headers (`##`)
   - Specific task descriptions
   - Contextual information
   - Step-by-step instructions
   - Structured output requirements

## API Reference

```python
from transform import (
    transform_with_intent_classification,  # Full pipeline (recommended)
    classify_intent,                        # Intent classification only
    transform_prompt,                       # Transformation only
    API_CAPABILITY_GUIDANCE,               # Available API capability flags
)

# Full pipeline
result = transform_with_intent_classification("Your prompt here")
print(result['intents'])      # Detected intents
print(result['nova_final'])   # Optimized prompt

# With API capability flags (for features you know you'll use)
result = transform_with_intent_classification(
    "Analyze the chart and recommend next steps",
    reasoning_mode=True,  # Using Nova reasoning mode API
    image=True           # Passing an image via API
)

# Just classification
intents = classify_intent("Your prompt")['intents']

# Transform with specific intents
result = transform_prompt("Your prompt", intents=['image_understanding'])

# Transform with API capabilities only (skip intent classification)
result = transform_prompt("Your prompt", intents=[], reasoning_mode=True, tool_use=True)
```